In [1]:
import pandas as pd
from schema.enums import HerdVariety
import os

### Initiate Dataframe to hold survey data

In [2]:
schema_path = os.path.join("schema", "input_schema_mapping.csv")
input_schema = pd.read_csv(schema_path)

# Build structured mapping
schema_dict = {
    row.metric: {
        "cell": row.survey_mapping,
        "type": row.types
    }
    for _, row in input_schema.iterrows()
}

# Extract columns
input_columns = list(schema_dict.keys())

In [3]:
from openpyxl import load_workbook

# Map your schema types to pandas dtypes
type_mapping = {
    "int": "Int64",        # nullable integer
    "float": "float64",
    "str": "string",
    "bool": "boolean",
    "string": "string"
}

# Build the dtype dictionary for pandas
dtypes = {
    col: type_mapping.get(schema_dict[col]["type"], "object")
    for col in input_columns
}

# Initialize empty DataFrame with correct dtypes
dff = pd.DataFrame({col: pd.Series(dtype=dtypes[col]) for col in input_columns})

print(dff.dtypes)


input_dir = "inputs"

for file_name in os.listdir(input_dir):
    if not file_name.lower().endswith(".xlsx"):
        continue

    print(file_name)

    file_path = os.path.join(input_dir, file_name)
    wb = load_workbook(file_path, data_only=True)
    ws = wb.active

    row_data = {}

    for metric, info in schema_dict.items():
        try:
            value = ws[info["cell"]].value
        except:
            value = None
        row_data[metric] = value

    dff = pd.concat([dff, pd.DataFrame([row_data])], ignore_index=True)

print(dff)

dff.to_csv("outputs/template.csv", index=False)


farm_id                         string[python]
general.grazing_area_ha                float64
milk_year                       string[python]
total_milk_production_litres           float64
milk_fat_content_percent                 Int64
                                     ...      
energy.diesel                          float64
energy.electricity                     float64
energy.renewables                      float64
energy.petrol                          float64
energy.gas                             float64
Length: 166, dtype: object
test_correct_inputs.xlsx
  farm_id  general.grazing_area_ha milk_year  total_milk_production_litres  \
0    test                    700.0      2025                        1000.0   

   milk_fat_content_percent  milk_protein_content_percent main_breed_variety  \
0                         5                             5          Fleckvieh   

   calf_dairy.herd_count  calf_dairy.herd_weight_kg  calf_dairy.sold_count  \
0                    100           

### Read data from excel dump into Data Frame

In [4]:
# ensure feeding piece is translated into days dmi based on their suggestions

feed_conversion_mapping = {
    "fwi_select": "C61",
    "dmi_select": "D61",
    "feed_per_animal": "C64",
    "feed_per_herd": "D64",
    "feed_period_day_single": "C67",
    "feed_period_day_custom": "D67"
}



### ID mappings

### ID mappings

In [5]:
id_mappings = {
    "grazing_quality": {
        "HIGH": 1,
        "LOW": 2,
        None: 1
    }
}

In [6]:
units = {
    "fertilizer_application_rate": 12, # t/ha
    "feed_weight": 9, # t
}

# make dry matter conversions

In [7]:
from config_loader import load_toml

feed_items = load_toml("feed.toml")["feed"]
herd_sections = load_toml("herd.toml")["herd_section"]
fertilizers = load_toml("fertilizer.toml")["fertilzier"]

In [8]:
for _, row in dff.iterrows():
    print(row)

farm_id                           test
general.grazing_area_ha          700.0
milk_year                         2025
total_milk_production_litres    1000.0
milk_fat_content_percent             5
                                 ...  
energy.diesel                   litres
energy.electricity                 kWh
energy.renewables                  kWh
energy.petrol                   litres
energy.gas                          M3
Name: 0, Length: 166, dtype: object


In [9]:
for _, row in dff.iterrows():


    # =========== Farm section ============

    farm_input = {
        "country": "Poland", 
        "territory": None,
        "climate": "Cool Temperate Moist",
        "average_temperature": {"value": 10, "unit": "°C"},
        "latitude": 52.679,
        "longitude": 20.030,
        "soil_characteristics": "Sandy Soils",
        "farm_identifier": row["farm_id"]
    }

    # =========== General section ============

    general_input = {
        "grazing_area": {
            "value": row["general.grazing_area_ha"],
            "unit": "ha"
        },
        "feed_approach": 1, # 1 = dmi
        "fertilizer_approach": 2, # 2 = Grazing, grass silage and hay area combined
    }

    # =========== Milk production section ============

    milk_production_input = {
        "variety": row["main_breed_variety"],
        "reporting_year": row["milk_year"],
        "date_time": "start",
        "date_month": 1, # Season always starts in Jan
        "name": row["farm_id"] + str("_") + str(row["milk_year"]),
        "product_dry": {"value": row["total_milk_production_litres"], "unit": "litres"},
        "fat_content": row["milk_fat_content_percent"],
        "protein_content": row["milk_protein_content_percent"],
        "protein_measure": 1 # 1 = true protein
    }

    # =========== Herd sections ============

    herd_sections_input = []

    for herd in herd_sections:
        herd_sections_input.append({
            "phase": herd["cft_name"],
            "animals": row[f"{herd['cft_name']}.herd_count"],
            "live_weight": {
                "value": row[f"{herd['cft_name']}.herd_weight_kg"],
                "unit": "kg"
            },
            "sold_animals": row[f"{herd['cft_name']}.sold_count"],
            "sold_weight": {
                "value": row[f"{herd['cft_name']}.sold_weight_kg"],
                "unit": "kg"
            },
            "purchased_animals": row[f"{herd['cft_name']}.purchased_count"],
            "purchased_weight": {
                "value": row[f"{herd['cft_name']}.purchased_weight_kg"],
                "unit": "kg"
            }
        })


    # =========== Grazing section ============

    grazing_input = []

    for herd in herd_sections:
        grazing_input.append({
            "herd_section": herd["cft_name"],
            "days": row[f"{herd['cft_name']}.grazing_days"],
            "hours": row[f"{herd['cft_name']}.grazing_hours_per_day"],
            "category": 2, # 2 = Confined pasture 
            "quality": id_mappings["grazing_quality"][row[f"{herd['cft_name']}.grazing_quality"]] # has to be int  1 =  high, 2 = low
        })



    # =========== Fertilizers section ============

    fertilizers_input = []

    for fertilizer in fertilizers:

        # if custom NPK
        if fertilizer["cft_id"] == 44:
            {
                "type": fertilizer["cft_id"], 
                "production": 8, # 8 = Europe 2014
                "custom_ingredients": {
                    "n_total_percentage": 6, # 6% as defined by Pawel
                    "n_ammonia_percentage": 6, # 6% as defined by Pawel
                    "p2o5_percentage": 20, # 20% as defined by Pawel
                    "p2o5_percentage_type_id": 4, # 4 = P2O5
                    "k2o_percentage": 30, # 30% as defined by Pawel
                    "k2o_percentage_type_id": 5 # 5 = K2O 
                },
                "application_rate": {
                    "value": row[f"fertilizers.{fertilizer['key']}.t_per_ha"],
                    "unit": units["fertilizer_application_rate"]
                },
                "application_date": "unknown",
                "rate_measure": "product",
                "inhibition": fertilizer["inhibition"] 
            },
        else:
            {
                "type": fertilizer["cft_id"], 
                "production": 8, # 8 = Europe 2014
                "application_rate": {
                    "value": row[f"fertilizers.{fertilizer['key']}.t_per_ha"],
                    "unit": units["fertilizer_application_rate"]
                },
                "application_date": "unknown",
                "rate_measure": "product",
                "inhibition": fertilizer["inhibition"]
            }


    # =========== Feed components section ============

    feed_components_input = []

    for feed in feed_items:
        
        for hs in herd_sections:
            feed_components_input.append({
                "item": feed["cft_id"],
                "region": feed["region_name"],
                "herd_section": hs["cft_name"],
                "dry_matter": {
                    "value": row[f"feed.{feed["cft_name"]}.{hs["cft_name"]}.tonne"],
                    "unit": units["feed_weight"]
                },
                "certified": False
            })

    # =========== Manure section ============

    manure_inputs = []

    herd_sections = ["calf_dairy", "heifer", "cow_milk", "cow_dry"]

    for herd in herd_sections:
        manure_type = row[f"manure_type.{herd}"] # names of columns
        
        if manure_type != "PIT STORAGE AND SOLID STORAGE":
            manure_inputs.extend([
                {"herd_section": herd, "type": 6, "allocation": 50},  # Pit Storage
                {"herd_section": herd, "type": 1, "allocation": 50},  # Solid Storage
            ])
        else:
            manure_inputs.append({
                "herd_section": herd,
                "type": manure_type,
                "allocation": 100
            })

    # =========== Compile dairy input ============

    dairy_input = {
        "farm": farm_input,
        "general": general_input,
        "milk_production": milk_production_input,
        "herd_sections": herd_sections_input,
        "grazing": grazing_input,
        "fertilizers": fertilizers_input,
        "feed_components": feed_components_input,
        "feed_additives": [], # leave empty as no additives
        "manure": manure_inputs,
        "direct_energy": [],
        "transport": []
    }

    print(dairy_input)

    # --- Call CFT API with dairy_input here --- #

KeyError: 'low'